# Загружаем библиотеки

In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import string

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 8.2MB 5.3MB/s 


In [4]:
from pymorphy2 import MorphAnalyzer

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import * 

# Читаем данные

[Датасет с твитами на русском языке](http://study.mokoron.com/) с разметкой *positive/negative*.

Читаем датафрейм с позитивными твитами, затем с негативными

In [6]:
tweets_positive = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/комп линг/tweets sentiment dataset/positive.csv',
                              sep=';', header=None, 
                              names=['id', 'date', 'author', 'text', 'type', 'replies', 'added_to_faves', 'author_tweets_count', 'retweets', 'followers','followed', 'lists'])

In [7]:
print(tweets_positive.shape)
tweets_positive['type'].unique()

(114911, 12)


array([1])

In [8]:
tweets_negative = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/комп линг/tweets sentiment dataset/negative.csv',
                              sep=';', header=None, 
                              names=['id', 'date', 'author', 'text', 'type', 'replies', 'added_to_faves', 'author_tweets_count', 'retweets', 'followers','followed', 'lists'])

In [9]:
print(tweets_negative.shape)
tweets_negative['type'].unique()

(111923, 12)


array([-1])

Датасеты очень большие (около 100 000 строк каждый), поэтому возьмем часть данных - по 25 000 твитов с позитивной и негативной меткой

In [72]:
# tweets = tweets_positive.head(25000).append(tweets_negative.head(25000))
tweets = tweets_positive.append(tweets_negative)

In [73]:
print(tweets.shape)
tweets['type'].unique()

(226834, 12)


array([ 1, -1])

Оставляем только столбцы *text* (текст твита) и *type* (индикатор разметки) - с ними будем работать.

In [74]:
df = tweets[['text', 'type']]
df.head()

,text,type
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [13]:
df.to_csv('tweets_dataset.csv')

# Предобработка

Очищаем от тегов других пользователей и лишних пробельных символов.

In [75]:
df['text'].replace('(RT )?@[\w]*:?','', regex=True, inplace = True)
df['text'].replace('\s',' ', regex=True, inplace = True)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,text,type
0,"хоть я и школота, но поверь, у нас то же само...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,Ну ты идиотка) я испугалась за тебя!!!,1
3,"""Кто то в углу сидит и погибает от голода, а ...",1
4,"Вот что значит страшилка :D Но блин,посмотрев...",1


In [76]:
corpus = df['text'].to_list()
corpus[:5]

[' хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)',
 'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D',
 ' Ну ты идиотка) я испугалась за тебя!!!',
 ' "Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим" :DD http://t.co/GqG6iuE2…',
 ' Вот что значит страшилка :D Но блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D']

Токенизация, приведение к нижнему регистру и удаление знаков препинания

In [16]:
punct_marks = string.punctuation + "—" + "«" + "»" + "`" + "``"

In [77]:
for i in range(len(corpus)):
  doc = corpus[i]
  doc = word_tokenize(doc)
  doc = [word.lower() for word in doc if word not in punct_marks]
  corpus[i] = doc

corpus[0]

['хоть',
 'я',
 'и',
 'школота',
 'но',
 'поверь',
 'у',
 'нас',
 'то',
 'же',
 'самое',
 'd',
 'общество',
 'профилирующий',
 'предмет',
 'типа']

Лемматизация и удаление стоп-слов: 
* предлогов
* союзов
* частиц
* междометий 
* местоимений

In [18]:
morph = MorphAnalyzer()

In [78]:
corpus_lemmatized = []

for i in range(len(corpus)):
  doc = corpus[i]
  doc_lemmatized = []
  for word in doc:
    p = morph.parse(word)[0]
    if p.tag.POS is None or \
      p.tag.POS in ('PREP', 'CONJ', 'PRCL', 'INTJ', 'NPRO') or \
      'Anph' in p.tag or 'Apro' in p.tag:
      continue
    doc_lemmatized.append(p.normal_form)
  corpus_lemmatized.append(doc_lemmatized)

In [79]:
corpus_lemmatized[:5]

[['школотый', 'общество', 'профилировать', 'предмет', 'тип'],
 ['немного', 'похожий', 'мальчик', 'хороший'],
 ['идиотка', 'испугаться'],
 ['угол',
  'сидеть',
  'погибать',
  'голод',
  'ещё',
  'порция',
  'взять',
  'уже',
  'жрать',
  'хотеть'],
 ['страшилка',
  'посмотреть',
  'часть',
  'создаться',
  'ощущение',
  'автор',
  'курить',
  'что-то']]

Сохраним лемматизированные документы в виде строк, чтобы их мог принять векторизатор

In [80]:
X = [' '.join(text) for text in corpus_lemmatized]
X[0]

'школотый общество профилировать предмет тип'

Разделим выборку на обучающую и тестовую

In [81]:
y = df['type']

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Мешок слов

Векторизуем тренировочные и тестовые данные

In [25]:
vectorizer = CountVectorizer(ngram_range=(1, 3))

In [26]:
X_train_vectorized = vectorizer.fit_transform(X_train)
list(vectorizer.vocabulary_.items())[:10]

[('завтра', 338865),
 ('очень', 715197),
 ('хороший', 1245491),
 ('день', 242975),
 ('писать', 751668),
 ('по', 764416),
 ('история', 407841),
 ('билет', 50434),
 ('биология', 51282),
 ('завтра очень', 340895)]

Обучим модель и выведем матрицу с оценками качества классификации

In [27]:
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [28]:
X_test_vectorized = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.72      0.73      0.72     28000
           1       0.73      0.73      0.73     28709

    accuracy                           0.73     56709
   macro avg       0.73      0.73      0.73     56709
weighted avg       0.73      0.73      0.73     56709



Обучим несколько моделей с разным диапазоном n и сохраним значения метрик качества в датафрейм

In [58]:
eval_data = {'vectorizer': [], 'n_range': [], 
             'min_df': [], 'max_df': [],
             'precision': [], 'recall': [],
             'accuracy': [], 'f_measure': []}

In [54]:
n_ranges = [(1, 1), (1, 3), (1, 4), (1, 5), (1, 6)]

In [55]:
max_df_list = [0.6, 0.75, 0.95, 0.98]
min_df_list = [1, 2, 3, 4]

In [59]:
for n_range in n_ranges:
  for max_df in max_df_list:
    for min_df in min_df_list:
      vectorizer = CountVectorizer(ngram_range=n_range, 
                                   min_df=min_df, max_df=max_df)
      X_train_vectorized = vectorizer.fit_transform(X_train)

      clf.fit(X_train_vectorized, y_train)

      X_test_vectorized = vectorizer.transform(X_test)
      y_pred = clf.predict(X_test_vectorized)

      precision = precision_score(y_test, y_pred)
      recall = recall_score(y_test, y_pred)
      accuracy = accuracy_score(y_test, y_pred)
      f_measure = f1_score(y_test, y_pred)

      eval_data['vectorizer'].append('ngrams')
      eval_data['n_range'].append(n_range)
      eval_data['min_df'].append(min_df)
      eval_data['max_df'].append(max_df)
      eval_data['precision'].append(precision)
      eval_data['recall'].append(recall)
      eval_data['accuracy'].append(accuracy)
      eval_data['f_measure'].append(f_measure)

# Tf-Idf векторизация

Повторим то же самое с использованием Tf-Idf векторизации

In [60]:
for n_range in n_ranges:
  for max_df in max_df_list:
    for min_df in min_df_list:
      tfidf_vectorizer = TfidfVectorizer(ngram_range=n_range, max_df = max_df, min_df = min_df)
      X_train_vectorized = tfidf_vectorizer.fit_transform(X_train)

      clf = MultinomialNB()
      clf.fit(X_train_vectorized, y_train)

      X_test_vectorized = tfidf_vectorizer.transform(X_test)
      y_pred = clf.predict(X_test_vectorized)

      precision = precision_score(y_test, y_pred)
      recall = recall_score(y_test, y_pred)
      accuracy = accuracy_score(y_test, y_pred)
      f_measure = f1_score(y_test, y_pred)

      eval_data['vectorizer'].append('tf-idf')
      eval_data['n_range'].append(n_range)
      eval_data['min_df'].append(min_df)
      eval_data['max_df'].append(max_df)
      eval_data['precision'].append(precision)
      eval_data['recall'].append(recall)
      eval_data['accuracy'].append(accuracy)
      eval_data['f_measure'].append(f_measure)

# Мешок символьных n-грамм

Аналогичным образом обучим модель с использованием символьных n-грамм

In [61]:
n_ranges = [(2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11)]

In [63]:
max_df_list = [0.6, 0.75, 0.95, 0.98]
min_df_list = [1, 2, 3, 4, 5]

In [64]:
for n_range in n_ranges:
  for max_df in max_df_list:
    for min_df in min_df_list:
      char_vectorizer = CountVectorizer(analyzer='char', ngram_range=n_range, min_df=min_df, max_df=max_df)
      X_train_vectorized = char_vectorizer.fit_transform(X_train)

      clf = MultinomialNB()
      clf.fit(X_train_vectorized, y_train)

      X_test_vectorized = char_vectorizer.transform(X_test)
      y_pred = clf.predict(X_test_vectorized)

      precision = precision_score(y_test, y_pred)
      recall = recall_score(y_test, y_pred)
      accuracy = accuracy_score(y_test, y_pred)
      f_measure = f1_score(y_test, y_pred)

      eval_data['vectorizer'].append('nchars')
      eval_data['n_range'].append(n_range)
      eval_data['min_df'].append(min_df)
      eval_data['max_df'].append(max_df)
      eval_data['precision'].append(precision)
      eval_data['recall'].append(recall)
      eval_data['accuracy'].append(accuracy)
      eval_data['f_measure'].append(f_measure)

# Сравнение метрик качества

Сохраним значения метрик качества в виде датафрейма

In [ ]:
eval_data = pd.DataFrame(eval_data)

In [94]:
eval_data

,vectorizer,n_range,min_df,max_df,precision,recall,accuracy,f_measure
0,ngrams,"(1, 1)",1,0.60,0.704115,0.686903,0.70264,0.695403
1,ngrams,"(1, 1)",2,0.60,0.695191,0.699692,0.70000,0.697434
2,ngrams,"(1, 1)",3,0.60,0.691975,0.699369,0.69760,0.695652
3,ngrams,"(1, 1)",4,0.60,0.687841,0.694188,0.69320,0.691000
4,ngrams,"(1, 1)",1,0.75,0.704115,0.686903,0.70264,0.695403
...,...,...,...,...,...,...,...,...
275,nchars,"(2, 11)",1,0.98,0.698138,0.685932,0.69824,0.691981
276,nchars,"(2, 11)",2,0.98,0.681157,0.728023,0.69720,0.703811
277,nchars,"(2, 11)",3,0.98,0.679286,0.714586,0.69224,0.696489
278,nchars,"(2, 11)",4,0.98,0.682931,0.700178,0.69120,0.691447


Выведем три лучших векторизатора по **F-мере**.  
Это оказались векторизаторы с параметрами:
* **мешок n-грамм**
* **n = (1, 6)**
* **min_df = 2**  

Значение **max_df**, по-видимому, не влияет на результат.  

In [66]:
eval_data.sort_values(by='f_measure', ascending=False).iloc[:3]

,vectorizer,n_range,min_df,max_df,precision,recall,accuracy,f_measure
73,ngrams,"(1, 6)",2,0.95,0.666348,0.790837,0.70096,0.723275
65,ngrams,"(1, 6)",2,0.60,0.666348,0.790837,0.70096,0.723275
77,ngrams,"(1, 6)",2,0.98,0.666348,0.790837,0.70096,0.723275


У векторизаторов наибольший recall, но относительно низкое значение precision.  
Выведем три лучших векторизатора по **precision**: 
* **Tf-Idf** 
* **n = (1, 6)**
* **min_df = 1**
* max_df не влияет

In [67]:
eval_data.sort_values(by='precision', ascending=False).iloc[:3]

,vectorizer,n_range,min_df,max_df,precision,recall,accuracy,f_measure
156,tf-idf,"(1, 6)",1,0.98,0.715729,0.688036,0.7108,0.70161
152,tf-idf,"(1, 6)",1,0.95,0.715729,0.688036,0.7108,0.70161
148,tf-idf,"(1, 6)",1,0.75,0.715729,0.688036,0.7108,0.70161


Напоследок выведем лидеров по набору параметров для каждого вида векторизатора

In [71]:
eval_data.groupby('vectorizer').apply(
    lambda x: x.sort_values(by='f_measure', ascending=False).head(1))

,,vectorizer,n_range,min_df,max_df,precision,recall,accuracy,f_measure
vectorizer,,,,,,,,,
nchars,266,nchars,"(2, 11)",2,0.75,0.681144,0.728671,0.69736,0.704106
ngrams,65,ngrams,"(1, 6)",2,0.60,0.666348,0.790837,0.70096,0.723275
tf-idf,149,tf-idf,"(1, 6)",2,0.75,0.678987,0.764287,0.70496,0.719117


# Обучение на всех данных 

На четверти данных удается получить значения метрик качества только около 0.7.  
Обучим модели с лучшими векторизаторами на полном датасете.

Векторизатор с лучшими F-мерой и recall

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 6), min_df=2)
X_train_vectorized = vectorizer.fit_transform(X_train)
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train)
X_test_vectorized = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_vectorized)

In [87]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.73      0.70      0.71     28010
           1       0.72      0.74      0.73     28699

    accuracy                           0.72     56709
   macro avg       0.72      0.72      0.72     56709
weighted avg       0.72      0.72      0.72     56709



Векторизатор с самым высоким precision

In [88]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 6), min_df = 1)
X_train_vectorized = vectorizer.fit_transform(X_train)
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train)
X_test_vectorized = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.73      0.70      0.71     28010
           1       0.72      0.74      0.73     28699

    accuracy                           0.72     56709
   macro avg       0.72      0.72      0.72     56709
weighted avg       0.72      0.72      0.72     56709



Лучший векторизатор среди мешков символьных n-грамм

In [89]:
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 11), min_df=2)
X_train_vectorized = vectorizer.fit_transform(X_train)
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train)
X_test_vectorized = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.73      0.70      0.71     28010
           1       0.72      0.74      0.73     28699

    accuracy                           0.72     56709
   macro avg       0.72      0.72      0.72     56709
weighted avg       0.72      0.72      0.72     56709



К сожалению, все три метрики качества все равно остаются около 0.7